In [1]:
#importing required libraries etc
import pandas as pd
import requests 

<h1> Part 1 </h1>

<h1> Scraping the Wiki page </h1>

In [2]:
wiki_link = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
wiki_page = requests.get(wiki_link)
wiki_doc = wiki_page.text

<h1> Reading the table with BeautifulSoup </h1>

In [4]:
from bs4 import BeautifulSoup
soup = BeautifulSoup(wiki_doc, 'html.parser')
table = soup.find('table', {'class': 'wikitable sortable'})

<h1> Converting the table to a pandas dataframe </h1>

In [5]:
col_names = ["PostalCode", "Borough", "Neighborhood"]
df = pd.read_html(str(table), skiprows=1)
df = pd.DataFrame.from_dict(df[0])
df.columns = col_names
df.head()

,PostalCode,Borough,Neighborhood
0,M2A,Not assigned,NaN
1,M3A,North York,Parkwoods
2,M4A,North York,Victoria Village
3,M5A,Downtown Toronto,"Regent Park, Harbourfront"
4,M6A,North York,"Lawrence Manor, Lawrence Heights"


<h1> Removing cells with no borough assigned </h1>

In [7]:
df = df[df.Borough != 'Not assigned'].reset_index(drop=True)
df.head()

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


<h1> Joining neighborhoods with the same postal code into one row, separating them by a comma </h1>

In [8]:
def combine_neighborhoods(series):
    return series.str.cat(sep=', ')

df_by_postcode = df.groupby(["PostalCode", "Borough"])
df = df_by_postcode.agg({'Neighborhood': combine_neighborhoods}).reset_index()
df.head()

,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,"Malvern, Rouge"
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


<h1> Assigning borough name as neighborhood name where the latter is missing </h1>

In [9]:
def impute_neighborhood(row):
    if row['Neighborhood'] == 'Not assigned':
        row['Neighborhood'] = row['Borough']
    
    return row

df = df.apply(impute_neighborhood, axis=1)
df.head()

,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,"Malvern, Rouge"
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


<h1> Total number of rows in the final dataframe </h1>

In [12]:
df.shape[0]

103

<h1> Part 2 </h1>

<h1> Loading coordinates </h1>

In [16]:
!pip install geocoder
import geocoder

     |████████████████████████████████| 102kB 7.3MB/s ta 0:00:011


In [17]:
def get_latlng(postal_code):
    lat_lng_coords = None
    #loop until we get all coordinates
    while(lat_lng_coords is None):
        g = geocoder.arcgis('{}, Toronto, Ontario'.format(postal_code))
        lat_lng_coords = g.latlng
    return lat_lng_coords

get_latlng('M5G')

[43.65607218800005, -79.38565318999997]

In [18]:
#to get postal code coordinates
postal_codes = df['PostalCode']
coords = [get_latlng(postal_code) for postal_code in postal_codes.tolist() ]

<h1> Adding lat and lng columns to the dataframe </h1>

In [19]:
df_coords = pd.DataFrame(coords, columns = ['Latitude', 'Longitude'])
df['Latitude'] = df_coords['Latitude']
df['Longitude'] = df_coords['Longitude']
df.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern, Rouge",43.808626,-79.189913
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.785779,-79.157368
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.765806,-79.185284
3,M1G,Scarborough,Woburn,43.771545,-79.218135
4,M1H,Scarborough,Cedarbrae,43.768791,-79.238813


In [20]:
#check a specific postal code in the df
df[df.PostalCode == 'M5G']

,PostalCode,Borough,Neighborhood,Latitude,Longitude
57,M5G,Downtown Toronto,Central Bay Street,43.656072,-79.385653
